In [1]:
# Import Libraries
import joblib
import numpy as np
import pandas as pd
import requests
import json

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Load Model Locally

In [2]:
# Load the trained model and scaler
model = joblib.load('../models/best_model_random_forest.joblib')
scaler = joblib.load('../models/scaler.joblib')

print(f"Model type: {type(model).__name__}")
print(f"Model loaded successfully!")

Model type: RandomForestClassifier
Model loaded successfully!


In [3]:
# Define feature names
FEATURE_NAMES = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'
]

print(f"Features: {FEATURE_NAMES}")

Features: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']


## 2. Local Prediction

In [4]:
# Sample patient data
sample_patient = {
    'age': 63,
    'sex': 1,          # Male
    'cp': 3,           # Chest pain type
    'trestbps': 145,   # Resting blood pressure
    'chol': 233,       # Cholesterol
    'fbs': 1,          # Fasting blood sugar > 120
    'restecg': 0,      # Resting ECG
    'thalach': 150,    # Max heart rate
    'exang': 0,        # Exercise induced angina
    'oldpeak': 2.3,    # ST depression
    'slope': 0,        # Slope of ST segment
    'ca': 0,           # Number of major vessels
    'thal': 1          # Thalassemia
}

# Convert to array
features = np.array([[sample_patient[f] for f in FEATURE_NAMES]])
print(f"Input features shape: {features.shape}")
print(f"Features: {features[0]}")

Input features shape: (1, 13)
Features: [ 63.    1.    3.  145.  233.    1.    0.  150.    0.    2.3   0.    0.
   1. ]


In [5]:
# Scale features and predict
features_scaled = scaler.transform(features)

prediction = model.predict(features_scaled)[0]
probability = model.predict_proba(features_scaled)[0]

print("\n" + "="*50)
print("LOCAL PREDICTION RESULT")
print("="*50)
print(f"Prediction: {'Heart Disease' if prediction == 1 else 'No Heart Disease'}")
print(f"Probability (No Disease): {probability[0]:.4f}")
print(f"Probability (Disease): {probability[1]:.4f}")
print(f"Confidence: {max(probability):.4f}")
print("="*50)

e:\Study\BITS PIlani\Sem 3\Assignments\MLOPS\heart-disease-mlops\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



LOCAL PREDICTION RESULT
Prediction: No Heart Disease
Probability (No Disease): 0.6974
Probability (Disease): 0.3026
Confidence: 0.6974


## 3. API Prediction (Kubernetes Deployment)

In [6]:
# API endpoint (Kubernetes LoadBalancer)
API_URL = "http://localhost:80"

# Check API health
try:
    response = requests.get(f"{API_URL}/health", timeout=5)
    print(f"API Health: {response.json()}")
except Exception as e:
    print(f"API not available: {e}")
    print("Make sure the Kubernetes deployment is running!")

API Health: {'status': 'healthy', 'model_loaded': True, 'timestamp': '2025-12-14T23:01:19.188972'}


In [7]:
# Make prediction via API
def predict_via_api(patient_data: dict, api_url: str = API_URL) -> dict:
    """
    Make a prediction using the deployed API.
    
    Args:
        patient_data: Dictionary with patient features
        api_url: Base URL of the API
    
    Returns:
        Dictionary with prediction results
    """
    response = requests.post(
        f"{api_url}/predict",
        headers={"Content-Type": "application/json"},
        json=patient_data,
        timeout=10
    )
    response.raise_for_status()
    return response.json()

# Test API prediction
try:
    result = predict_via_api(sample_patient)
    
    print("\n" + "="*50)
    print("API PREDICTION RESULT")
    print("="*50)
    print(f"Prediction: {result['prediction_label']}")
    print(f"Probability (No Disease): {result['probability_no_disease']:.4f}")
    print(f"Probability (Disease): {result['probability_disease']:.4f}")
    print(f"Confidence: {result['confidence']:.4f}")
    print(f"Model Version: {result['model_version']}")
    print(f"Timestamp: {result['timestamp']}")
    print("="*50)
    
except Exception as e:
    print(f"API Error: {e}")


API PREDICTION RESULT
Prediction: No Heart Disease
Probability (No Disease): 0.6974
Probability (Disease): 0.3026
Confidence: 0.6974
Model Version: 1.0.0
Timestamp: 2025-12-14T23:01:19.308282


## 4. Batch Predictions

In [8]:
# Multiple patients for batch prediction
test_patients = [
    {"age": 45, "sex": 0, "cp": 1, "trestbps": 120, "chol": 200, "fbs": 0, 
     "restecg": 1, "thalach": 170, "exang": 0, "oldpeak": 0.5, "slope": 1, "ca": 0, "thal": 2},
    {"age": 67, "sex": 1, "cp": 3, "trestbps": 160, "chol": 280, "fbs": 1, 
     "restecg": 2, "thalach": 120, "exang": 1, "oldpeak": 3.5, "slope": 2, "ca": 2, "thal": 3},
    {"age": 52, "sex": 1, "cp": 2, "trestbps": 140, "chol": 220, "fbs": 0, 
     "restecg": 0, "thalach": 155, "exang": 0, "oldpeak": 1.0, "slope": 1, "ca": 0, "thal": 2},
]

print("Batch Prediction Results:")
print("="*70)

for i, patient in enumerate(test_patients, 1):
    try:
        result = predict_via_api(patient)
        print(f"Patient {i}: Age={patient['age']}, Sex={'M' if patient['sex']==1 else 'F'} -> "
              f"{result['prediction_label']} (confidence: {result['confidence']:.2f})")
    except Exception as e:
        # Fallback to local prediction
        features = np.array([[patient[f] for f in FEATURE_NAMES]])
        features_scaled = scaler.transform(features)
        pred = model.predict(features_scaled)[0]
        prob = model.predict_proba(features_scaled)[0]
        label = 'Heart Disease' if pred == 1 else 'No Heart Disease'
        print(f"Patient {i}: Age={patient['age']}, Sex={'M' if patient['sex']==1 else 'F'} -> "
              f"{label} (confidence: {max(prob):.2f}) [LOCAL]")

Batch Prediction Results:
Patient 1: Age=45, Sex=F -> No Heart Disease (confidence: 0.96)
Patient 2: Age=67, Sex=M -> Heart Disease (confidence: 0.77)
Patient 3: Age=52, Sex=M -> No Heart Disease (confidence: 0.93)


## 5. Model Information

In [9]:
# Display model information
print("\nModel Information:")
print("="*50)
print(f"Model Type: {type(model).__name__}")
print(f"Number of Features: {len(FEATURE_NAMES)}")
print(f"Feature Names: {FEATURE_NAMES}")

if hasattr(model, 'n_estimators'):
    print(f"Number of Estimators: {model.n_estimators}")
if hasattr(model, 'max_depth'):
    print(f"Max Depth: {model.max_depth}")

print("="*50)


Model Information:
Model Type: RandomForestClassifier
Number of Features: 13
Feature Names: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
Number of Estimators: 100
Max Depth: 10


In [10]:
# Feature importance
if hasattr(model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'Feature': FEATURE_NAMES,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nFeature Importance:")
    print(importance_df.to_string(index=False))


Feature Importance:
 Feature  Importance
    thal    0.156102
      cp    0.132607
 thalach    0.123178
      ca    0.120167
 oldpeak    0.099517
     age    0.076773
    chol    0.067624
trestbps    0.058463
   exang    0.055311
   slope    0.045294
     sex    0.039927
 restecg    0.018421
     fbs    0.006617


## Summary

This notebook demonstrated:
1. ✅ Loading trained model locally
2. ✅ Making predictions with preprocessing
3. ✅ Calling the deployed Kubernetes API
4. ✅ Batch predictions for multiple patients

The model is ready for production use!